In [1]:
import nltk

# nltk.download('stopwords')

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [3]:
# Gensim (Topic Modeling Pacakge)
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [4]:
# spacy for lemmatization
import spacy

from spacy_download import load_spacy

In [5]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [7]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [8]:
'''
We will be using the 20-Newsgroups dataset for this exercise.
This version of the dataset contains about 11k newsgroups posts from 
20 different topics. This is available as newsgroups.json.
'''
# Import Dataset
df = pd.read_csv('data/post_covidTweets_v2.csv')
print(df.username.unique())
df.head()
df.count()

['RochelleAdame7' 'brian_tietz' 'LarryBoorstein' 'VigilantFox'
 'Mrmangajojo' 'HawkInATX' 'vindicator1337' 'KoonsKarl' 'JohnnyRode'
 'Oneguy83440339' 'Excuseeeme' 'StuckInSanDiego' 'JudithArnett1'
 'Tigerlily1011' 'Deevoee' 'Katetheskates' 'LankeyPants250'
 'FellwockPerry' 'most_articulate' 'Smpwns' 'USBornNRaised' 'AlisynGayle'
 'RobinBr83683748' 'urbiasdqsu' 'adelaidians' 'mrmikebones' '_LoveLiberty'
 'LonelyFreedom' 'ClimateDepot' 'CharmsYahoo' 'STRAT777' 'Salty_shifter'
 'JamieO123' 'Faketriots' 'CristinoCastro9' '76Americanwomen'
 'io_aircraft_inc' 'MrMarchi' 'KingGrunt' 'MrSkanner' 'PatrioticPatty2'
 'SherryHighHorse' 'DailyReformerUS' 'TruckCastreauJr' 'valentino_nora'
 'OrmeStephan' 'JohnT43838185' 'Kellyeod' 'DennisOgden007' 'Alec121212'
 'JackHay01988146' 'small4lyfe' 'Allison86435394' 'nomandates23'
 'andriedeja' 'ImmunifyLife' 'tomcoates' 'caz_sampson' 'Crof'
 'Amfirstnewswire' 'WoodyWhitcomb' 'GeorgeG55508748' 'livejelly'
 'BuzzerlyApp' 'kyle8824' 'stephphilip8' 'infamous_

datetime          40886
tweet_id          40886
text              40886
username          40886
url               40886
user_location     27639
like_count        40886
retweet_count     40886
follower_count    40886
reply_count       40886
verified          40886
hashtags          14700
dtype: int64

In [9]:
'''
As you can see there are many emails, newline and extra spaces that is quite distracting.
Let’s get rid of them using regular expressions.
'''
# Convert to list
data = df.text.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

#remove URL
data = [re.sub('http://\S+|https://\S+', '', sent) for sent in data]
data = [re.sub('http[s]?://\S+', '', sent) for sent in data]
data = [re.sub(r"http\S+", "", sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['Did I say it was evil? Did I say dont get it? NEWSFLASH....I DIDNT. And I '
 'get the flu shot every year and my mammogram in the same week until '
 'recently. I have never been told I had to postpone my mammogram for any '
 'other vaccine. NOT ONE.']


In [10]:

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['did', 'say', 'it', 'was', 'evil', 'did', 'say', 'dont', 'get', 'it', 'newsflash', 'didnt', 'and', 'get', 'the', 'flu', 'shot', 'every', 'year', 'and', 'my', 'mammogram', 'in', 'the', 'same', 'week', 'until', 'recently', 'have', 'never', 'been', 'told', 'had', 'to', 'postpone', 'my', 'mammogram', 'for', 'any', 'other', 'vaccine', 'not', 'one']]


In [11]:

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['did', 'say', 'it', 'was', 'evil', 'did', 'say', 'dont', 'get', 'it', 'newsflash', 'didnt', 'and', 'get', 'the', 'flu', 'shot', 'every', 'year', 'and', 'my', 'mammogram', 'in', 'the', 'same', 'week', 'until', 'recently', 'have', 'never', 'been', 'told', 'had', 'to', 'postpone', 'my', 'mammogram', 'for', 'any', 'other', 'vaccine', 'not', 'one']


In [12]:

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
# Will download the model if it isn't installed yet 
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])



print(data_lemmatized[:1])

[['say', 'evil', 'say', 'get', 'newsflash', 'get', 'flu', 'shoot', 'year', 'mammogram', 'week', 'recently', 'never', 'tell']]


In [14]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1)]]


In [15]:
vocab = len(id2word.token2id)

pd.Series([t for tokens in data_words_bigrams for t in tokens if "_" in t]).unique()

array(['lab_leak', 'bipartisan_investigation', 'spike_proteins', ...,
       'radio_canada', 'scoping_review', 'government_decision'],
      dtype=object)

In [16]:

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [17]:

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]



[(0,
  '0.287*"covid" + 0.061*"vaccine" + 0.020*"patient" + 0.016*"say" + '
  '0.014*"health" + 0.014*"know" + 0.013*"year" + 0.011*"risk" + 0.011*"take" '
  '+ 0.010*"make"'),
 (1,
  '0.042*"many" + 0.031*"good" + 0.029*"give" + 0.029*"post" + '
  '0.025*"vaccinate" + 0.020*"news" + 0.020*"social" + 0.018*"back" + '
  '0.018*"help" + 0.016*"read"'),
 (2,
  '0.106*"pandemic" + 0.044*"use" + 0.043*"impact" + 0.032*"response" + '
  '0.026*"policy" + 0.025*"bad" + 0.022*"catch" + 0.019*"claim" + '
  '0.016*"coronavirus" + 0.014*"push"'),
 (3,
  '0.053*"think" + 0.035*"stop" + 0.035*"month" + 0.031*"system" + '
  '0.029*"right" + 0.028*"chinese" + 0.028*"thing" + 0.025*"problem" + '
  '0.022*"protection" + 0.019*"old"'),
 (4,
  '0.054*"follow" + 0.043*"misinformation" + 0.033*"mrna" + 0.030*"medium" + '
  '0.026*"develop" + 0.026*"blood" + 0.025*"school" + 0.023*"add" + '
  '0.022*"viral" + 0.020*"create"'),
 (5,
  '0.101*"death" + 0.069*"die" + 0.050*"still" + 0.049*"datum" + '
  '0.041*"

In [18]:
'''
Model perplexity and topic coherence provide a convenient measure to judge 
how good a given topic model is. In my experience, topic coherence score, 
in particular, has been more helpful.
'''
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)




Perplexity:  -8.555564449247727

Coherence Score:  0.31482153598027657


In [19]:
'''
Now that the LDA model is built, the next step is to examine the produced 
topics and the associated keywords. There is no better tool than pyLDAvis package’s 
interactive chart and is designed to work well with jupyter notebooks.
'''
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\hannah.kim\AppData\Roaming\Python\Python39\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.350267 -0.289415       1        1  33.330506
6     -0.258993  0.370576       2        1  20.806299
1      0.072907 -0.009723       3        1   7.144531
2      0.045623 -0.022968       4        1   6.780044
5      0.077739 -0.007441       5        1   6.649332
8      0.053208 -0.016944       6        1   6.380549
7      0.075494 -0.007911       7        1   6.200156
9      0.091956 -0.004539       8        1   4.952580
3      0.089459 -0.003873       9        1   4.684684
4      0.102873 -0.007762      10        1   3.071319, topic_info=           Term          Freq         Total Category  logprob  loglift
30        covid  46013.000000  46013.000000  Default  30.0000  30.0000
64      vaccine   9744.000000   9744.000000  Default  29.0000  29.0000
339    pandemic   3448.000000   3448.000000  Default  28.0000  28.0000
123       death   3232.000000   3232.000000  Default  27.0000  27.0000
148        case   2424.000000   2424.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
2074       lack    161.325822    162.451975  Topic10  -4.5128   3.4761
653        half    160.260573    161.386663  Topic10  -4.5194   3.4761
1806    promise    150.158565    151.284777  Topic10  -4.5845   3.4756
2363  statement    149.436564    150.562709  Topic10  -4.5893   3.4756
213       drive    144.936897    146.063235  Topic10  -4.6199   3.4753

[331 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
532       3  0.997596        actually
787      10  0.997244             add
650      10  0.994034  administration
1671      9  0.996093           admit
718       4  0.997847           adult
...     ...       ...             ...
102       2  0.998788            well
1358      7  0.995618           woman
201       8  0.998808            work
48        9  0.998038           wrong
11        1  0.999270            year

[302 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 2, 3, 6, 9, 8, 10, 4, 5])

In [20]:
output = lda_model.show_topics(num_words=30)

In [21]:
output_df = pd.DataFrame()
for o in output:    
    d = {}
    topic_num = o[0]
    distribution = o[1]
    decimal_list = re.findall(r'\d+\.\d+', distribution)
    word_list = re.findall(r'"([^"]*)"', distribution)
    d["topic_" + str(topic_num + 1) + "_word"] = word_list
    d["topic_" + str(topic_num + 1) + "_dist"] =  decimal_list

    topic_df = pd.DataFrame(d)
    output_df = pd.concat([output_df, topic_df], axis=1)

output_df

,topic_1_word,topic_1_dist,topic_2_word,topic_2_dist,topic_3_word,topic_3_dist,topic_4_word,topic_4_dist,topic_5_word,topic_5_dist,topic_6_word,topic_6_dist,topic_7_word,topic_7_dist,topic_8_word,topic_8_dist,topic_9_word,topic_9_dist,topic_10_word,topic_10_dist
0,covid,0.287,many,0.042,pandemic,0.106,think,0.053,follow,0.054,death,0.101,study,0.040,pfizer,0.051,report,0.056,case,0.102
1,vaccine,0.061,good,0.031,use,0.044,stop,0.035,misinformation,0.043,die,0.069,people,0.036,s,0.036,sars_cov,0.048,virus,0.058
2,patient,0.020,give,0.029,impact,0.043,month,0.035,mrna,0.033,still,0.050,get,0.035,also,0.026,base,0.038,work,0.052
3,say,0.016,post,0.029,response,0.032,system,0.031,medium,0.030,datum,0.049,infection,0.024,come,0.025,public,0.028,much,0.036
4,health,0.014,vaccinate,0.025,policy,0.026,right,0.029,develop,0.026,hospital,0.041,long,0.021,research,0.023,change,0.021,start,0.028
5,know,0.014,news,0.020,bad,0.025,chinese,0.028,blood,0.026,lie,0.033,go,0.020,associate,0.023,look,0.020,real,0.026
6,year,0.013,social,0.020,catch,0.022,thing,0.028,school,0.025,kill,0.026,new,0.018,rate,0.021,evidence,0.019,early,0.023
7,risk,0.011,back,0.018,claim,0.019,problem,0.025,add,0.023,state,0.024,vaccination,0.017,re,0.019,include,0.016,question,0.022
8,take,0.011,help,0.018,coronavirus,0.016,protection,0.022,viral,0.022,experience,0.023,end,0.014,symptom,0.018,human,0.016,positive,0.020
9,make,0.010,read,0.016,push,0.014,old,0.019,create,0.020,vax,0.023,time,0.014,flu,0.017,survey,0.015,result,0.020
